In [1]:
from tensorflow.keras.models import load_model
import tensorflow_hub as hub

loaded_model = load_model('/kaggle/input/effnet-2/model')
           
#loaded_model = load_model('/kaggle/input/effnetv2-0-56/model_checkpoint.h5', custom_objects={'KerasLayer':hub.KerasLayer})

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
!ls /kaggle/input/pyvips-python-and-deb-package-gpu
# intall the deb packages
!yes | dpkg -i --force-depends /kaggle/input/pyvips-python-and-deb-package-gpu/linux_packages/archives/*.deb
# install the python wrapper
!pip install pyvips -f /kaggle/input/pyvips-python-and-deb-package-gpu/python_packages/ --no-index

linux_packages	python_packages
Selecting previously unselected package apparmor.
(Reading database ... 114840 files and directories currently installed.)
Preparing to unpack .../apparmor_3.0.4-2ubuntu2.2_amd64.deb ...
Unpacking apparmor (3.0.4-2ubuntu2.2) ...
Selecting previously unselected package autoconf.
Preparing to unpack .../autoconf_2.71-2_all.deb ...
Unpacking autoconf (2.71-2) ...
Selecting previously unselected package automake.
Preparing to unpack .../automake_13a1.16.5-1.3_all.deb ...
Unpacking automake (1:1.16.5-1.3) ...
Selecting previously unselected package autotools-dev.
Preparing to unpack .../autotools-dev_20220109.1_all.deb ...
Unpacking autotools-dev (20220109.1) ...
Selecting previously unselected package bzip2-doc.
Preparing to unpack .../bzip2-doc_1.0.8-5build1_all.deb ...
Unpacking bzip2-doc (1.0.8-5build1) ...
Selecting previously unselected package file.
Preparing to unpack .../file_13a5.41-3ubuntu0.1_amd64.deb ...
Unpacking file (1:5.41-3ubuntu0.1) ...
Sele

In [3]:
import os 
os.environ['VIPS_CONCURRENCY'] = '4'
os.environ['VIPS_DISC_THRESHOLD'] = '15gb'

In [4]:
import cv2
import numpy as np
from PIL import Image

def sampleimagest(image, size, n):
    
  images = [] 
    
  satt = 50
  wt = 220
  bt = 5

  scoret = 0.5
  rt = 0
    
  #50 210 5 0.7 worked well
  #60 215 5 0.5 also

  pits = 300
  its = int(20*n)

  images = []  
  scores = []
  
  redavgs = []
 
  a = 0
  b = 0

  #start = time.time()
    
  tma = False
    
  if image.width < 5000 and image.height < 5000:
    tma = True 
    size = size*2
    its = int(3*n)
    n = int(n/2)

  while(a < pits):
    
    x = np.random.randint(image.width-size)
    y = np.random.randint(image.height-size)
    
    #t0 = time.time()
    im = image.crop(x, y, size, size).numpy()
    
    #if (a == 0):
    #   print("timec: " + str(time.time() - t0))
    
    red = im[:,:,0]/255.0
    blue = im[:,:,2]/255.0
    
    hls = cv2.cvtColor(im, cv2.COLOR_RGB2HLS)
        
    h, l, s = hls[..., 0], hls[..., 1], hls[..., 2]
    
    nob = l > bt
    now = l < wt
    
    lscore = np.sum(nob & now)/(size * size) 
    goodpix = nob & now 
    
    if lscore < scoret:
        continue
    
    redavgs.append(np.mean(red - blue))
    a += 1
    
  redavg = np.mean(redavgs)
  redstd = np.std(redavgs)
  
  #print("pits: " + str(time.time() - start))
  #start = time.time()
  while b < its:

    x = np.random.randint(image.width-size)
    y = np.random.randint(image.height-size)
    
    #t0 = time.time()
    im = image.crop(x, y, size, size).numpy()
    #if (b == 0):
    #   print("timec: " + str(time.time() - t0))
    
    #im = image[x:x+size,y:y+size,:]
    hls = cv2.cvtColor(im, cv2.COLOR_RGB2HLS)
        
    h, l, s = hls[..., 0], hls[..., 1], hls[..., 2]
    
    nob = l > bt
    now = l < wt
    sat = s > satt
    
    lscore = np.sum(nob & now)/(size * size) 
    goodpix = nob & now & sat
    
    score = np.sum(goodpix)/(size * size)  
    
    red = im[:,:,0]/255.0
    blue = im[:,:,2]/255.0
    
    #MAKE STD AND AVG COLOR 
    
    diff = np.mean(red - blue) 
    redness = (diff - redavg) / redstd
    
    if redness > rt:
      continue
    
    b += 1 
    
    '''if tma:
        images.append(cv2.resize(im, (int(size/2),int(size/2)), interpolation = cv2.INTER_AREA))
    else:
        images.append(im)'''  
    
    images.append(im)
    scores.append(score)
    
  s = np.argsort(scores)[::-1]

  #print("its: " + str(time.time() - start))
    
  ret = []

  for i in range(n):
    if tma:
        ret.append(np.array(Image.fromarray(images[s[i]]).resize((int(size/2),int(size/2)), Image.LANCZOS)))
    else:
    
        ret.append(images[s[i]])
  
  return ret 

In [5]:
class2label = {"CC": 0, "EC": 1, "HGSC": 2, "LGSC": 3, "MC": 4}
label2class = {0: "CC", 1: "EC", 2: "HGSC", 3: "LGSC", 4: "MC"}

In [6]:
import pandas as pd
import numpy as np
from scipy import stats
import pyvips
import time
import gc
from multiprocessing.pool import ThreadPool


os.environ['CUDA_VISIBLE_DEVICES'] = '0'


def process_image(image_info):
    start = time.time()
    dirname, filename, n = image_info
    image_path = os.path.join(dirname, filename)
    fd = filename.split(".")[0]

    image = pyvips.Image.new_from_file(image_path)
    imgs = np.array(sampleimagest(image, 224, n)) / 255.0

    pred = loaded_model.predict(imgs)
    mask = pred > 0.99
    
    labels = np.argmax(pred, axis=-1)
    truelabels = []
    
    label = 'Other'
    
    for i in range(len(labels)):
        if mask[i][labels[i]]:
            truelabels.append(labels[i])
    
    if len(truelabels) > 0:
        result = stats.mode(truelabels)
        label = label2class[result.mode[0]]
                
    # print(result.count[0]/n)
    #print(time.time() - start)
    return fd, label

In [7]:
#image = pyvips.Image.new_from_file('/kaggle/input/UBC-OCEAN/train_images/13568.png')
#imgs = np.array(sampleimagest(image, 224, 64))

In [8]:
submission = pd.read_csv('/kaggle/input/UBC-OCEAN/sample_submission.csv')
n = 64
#threshold = 0.3
image_info_list = []

for dirname, _, filenames in os.walk('/kaggle/input/UBC-OCEAN/test_images'):
    for filename in filenames:
        image_info_list.append((dirname, filename, n))

# =pool of workers 
with ThreadPool(processes=2) as pool:  
    results = pool.map(process_image, image_info_list)

s = time.time()
for idx, (image_id, label) in enumerate(results):
    submission.loc[idx, 'image_id'] = image_id
    submission.loc[idx, 'label'] = label
    gc.collect()



2/2 [==============================] - 7s 31ms/step


/tmp/ipykernel_26/1437816643.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '41' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  submission.loc[idx, 'image_id'] = image_id


In [9]:
#display(submission)

In [10]:
submission.to_csv('submission.csv', index=None)